In [1]:
    try:
        import commonmark as CommonMark
        from commonmark import Parser
        from commonmark.render.renderer import Renderer
    except:
        import CommonMark
        from CommonMark import Parser
        from CommonMark.render.renderer import Renderer
    from textwrap import indent, dedent
    from importnb import Notebook
    from collections import UserList
    from abc import abstractmethod, ABCMeta
    try:
        from IPython.display import display, Markdown, HTML
        from IPython.core.inputtransformer import InputTransformer
        from IPython import get_ipython
    except:
        class InputTransformer:
            def __init__(self, *args, **kwargs): ...
            def transform_cell(self, str): return dedent(str)
            
        def get_ipython(): ...

## Markdown to code

Markdown is converted to code by persisting all code_blocks and replacing all non-code lines with blank lines.  The blank line replacement assures the right line numbering.

* Create a custom CommonMark renderer for code only.  It maintains the lines and positions of the original source to give better error messages.

* The commonmark renderer only catches code cells

    > There is an interesting discussion to be had about the role of inline cells.

Code fences can't be mixed with indented code at this point.

In [7]:
    class CodeRenderer(Renderer):
        def code(self, node, entering):
            if self.user_expressions is not None: 
                self.user_expressions += node.literal,
                
        def code_block(self, node, entering):
            
            spaces = ' '*(node.sourcepos[0][1])

            while len(self.buf.splitlines()) <  node.sourcepos[0][0]-1:
                self.out("\n")
            if node.is_fenced:
                self.out(indent("{}{}\n".format('%%' if node.info else '', node.info), spaces))
            self.out(indent(node.literal, spaces))
            
        repr = staticmethod(Markdown)
            
        def __call__(self, str, *user_expressions):
            self.user_expressions = user_expressions
            parsed = dedent(self.render(Parser().parse(str)))
            stripped = parsed.lstrip()
            if stripped.startswith('%%'):
                parsed = stripped

            return parsed, self.user_expressions

    renderer = CodeRenderer()

In [8]:
    class MarkdownImporter(Notebook):
        extensions = '.md.ipynb',
        def format(self, str): 
            source, user_expressions = renderer(str)
            return super().format(source)

In [9]:
    try:
        from IPython.display import HTML, Javascript
        
        style = HTML("""<style>
        .code-cell-toggle {
            position: fixed;
            bottom: 20px;
            right: 20px;
        }
        .code-cell-hidden .code_cell .input {
            display: none;
        }
        </style>""")

        toggle = Javascript("""$("button.code-cell-toggle").remove();
        var btn = $("<button/>", {"class": "code-cell-toggle"}).text("</>");
        btn.bind("click", function(){
            $("body").toggleClass("code-cell-hidden");
        });
        $("#notebook").append(btn);
        if (!window.location.host.startsWith('localhost')){
            $("body").addClass("code-cell-hidden");
        };""")
    except: ...

In [7]:
    class Test(__import__('unittest').TestCase): 
        def setUp(Test):
            %reload_ext pidgin
            load_ipython_extension()
            from nbformat import write, v4
            with open('test_markdown.md.ipynb', 'w') as file:
                write(v4.new_notebook(cells=[v4.new_code_cell("""Some paragraph\n\n    a=42""")]), file)
                
        def runTest(Test):
            global test_markdown
            import test_markdown
            assert test_markdown.__file__.endswith('.ipynb')
            assert test_markdown.a is 42

In [8]:
    if __name__ == '__main__':
        %reload_ext pidgin

### if __name__ == '__main__': 
        __import__('unittest').TextTestRunner().run(Test())